# Preliminary Preprocessing and modelling
The main goal of this notebook is to start wrintg the requiered steps/functions, requiered for the preprocessing of the data.

I will tray to:
* Impute the raw data with the columns median ( recall that most distributions are skewed ) and Run LazyPredict
* Impute the processed data ( long format ) with column meadian or time series interpolation and run LazyPredict

### Commentary on installing and LazyPredict
You can install LazyPredict to our project enviroment by running `pip install lazypredict==0.2.12`. Before using LazyPredict you have to make a slight modification. For regression, LazyPredict works by running ca. 40 regression models from sklearn. The problem, at least in my Machine, is that two of them  (`GaussianProcessRegressor`& `KernelRidge`) were the notebook's kernel and LazyPredict could not resume. For that reason, I removed them from the list of Regressors that. To do this, you would have to open this file `/My/AWESOME/PATH/.venv/lib/python3.12/site-packages/lazypredict/Supervised.py`, where `.venv` is the location of the virtual enviroment we use for our project. Once you open `Supervised.py`, go to line 53 and add "GaussianProcessRegressor" "KernelRidge".

Old list:
removed_regressors = [
    "TheilSenRegressor",
    "ARDRegression", 
    "CCA", 
    "IsotonicRegression", 
    "StackingRegressor",
    "MultiOutputRegressor", 
    "MultiTaskElasticNet", 
    "MultiTaskElasticNetCV", 
    "MultiTaskLasso", 
    "MultiTaskLassoCV", 
    "PLSCanonical", 
    "PLSRegression", 
    "RadiusNeighborsRegressor", 
    "RegressorChain", 
    "VotingRegressor", 
]

New list:
removed_regressors = [
    "TheilSenRegressor",
    "ARDRegression", 
    "CCA", 
    "IsotonicRegression", 
    "StackingRegressor",
    "MultiOutputRegressor", 
    "MultiTaskElasticNet", 
    "MultiTaskElasticNetCV", 
    "MultiTaskLasso", 
    "MultiTaskLassoCV", 
    "PLSCanonical", 
    "PLSRegression", 
    "RadiusNeighborsRegressor", 
    "RegressorChain", 
    "VotingRegressor", 
    "GaussianProcessRegressor",
    "KernelRidge"
]

Save the changes, and then you are ready to use LazyRegressor with the data from our project

### Approach 1: Imputing the RawData + LazyPRedict

In [1]:
# import requiered libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Hide all warnings
import warnings
warnings.filterwarnings( "ignore" )

In [2]:
# Read the raw data
train = pd.read_csv( '../data/raw/train.csv' )
train.head()

,id,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,...,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
0,p01_0,p01,06:10:00,NaN,NaN,9.6,NaN,NaN,9.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.4
1,p01_1,p01,06:25:00,NaN,NaN,9.7,NaN,NaN,9.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.8
2,p01_2,p01,06:40:00,NaN,NaN,9.2,NaN,NaN,8.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.5
3,p01_3,p01,06:55:00,NaN,NaN,8.7,NaN,NaN,8.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.8
4,p01_4,p01,07:10:00,NaN,NaN,8.4,NaN,NaN,8.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7


At the beginning, we will remove the columns related to carbs and activity, as they have very large ammount of NAs

In [3]:
cond = ( train.columns.str.startswith('activity-') ) | ( train.columns.str.startswith('carbs-') )
exclude_cols = train.columns[ np.where( cond )[0] ]

# Also exclude the 'id','p_num' & 'time'
train_s = train.drop( exclude_cols , axis  = 1 ).drop( ['id','p_num','time'] , axis = 1)
train_s

,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,bg-5:20,bg-5:15,bg-5:10,...,cals-0:40,cals-0:35,cals-0:30,cals-0:25,cals-0:20,cals-0:15,cals-0:10,cals-0:05,cals-0:00,bg+1:00
0,NaN,NaN,9.6,NaN,NaN,9.7,NaN,NaN,9.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.4
1,NaN,NaN,9.7,NaN,NaN,9.2,NaN,NaN,8.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.8
2,NaN,NaN,9.2,NaN,NaN,8.7,NaN,NaN,8.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.5
3,NaN,NaN,8.7,NaN,NaN,8.4,NaN,NaN,8.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.8
4,NaN,NaN,8.4,NaN,NaN,8.1,NaN,NaN,8.3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177019,8.8,9.1,9.2,9.4,9.8,10.2,10.4,10.3,10.1,10.0,...,6.75,6.75,6.75,6.75,6.75,6.75,6.75,6.75,6.75,11.1
177020,9.1,9.2,9.4,9.8,10.2,10.4,10.3,10.1,10.0,9.8,...,6.75,6.75,6.75,6.75,6.75,6.75,6.75,6.75,6.75,10.9
177021,9.2,9.4,9.8,10.2,10.4,10.3,10.1,10.0,9.8,9.5,...,6.75,6.75,6.75,6.75,6.75,6.75,6.75,6.75,6.75,10.7
177022,9.4,9.8,10.2,10.4,10.3,10.1,10.0,9.8,9.5,9.2,...,6.75,6.75,6.75,6.75,6.75,6.75,6.75,6.75,6.75,10.5


Next, we create a train/test split. Then, impute the Train and test set with the medians of the train set

In [4]:
X = train_s.drop( 'bg+1:00' , axis = 1 )
y = train_s['bg+1:00']

X_train , X_test , y_train , y_test = train_test_split( X , y , test_size = .3 , random_state = 42 )

# Impute the columns of X_train and X_test
medians = X_train.median( axis = 0 )
X_train = X_train.fillna( medians )
X_test = X_test.fillna( medians )

Run a LazyPredict regressor. This will test 40 regression models from sklearn, using default parameters

In [5]:
from lazypredict.Supervised import LazyRegressor
reg = LazyRegressor( verbose = 0 , ignore_warnings = False , custom_metric = None )
models , predictions = reg.fit( X_train , X_test , y_train , y_test )

 98%|█████████▊| 39/40 [3:19:35<06:36, 396.20s/it]   

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.050561 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 88001
[LightGBM] [Info] Number of data points in the train set: 123916, number of used features: 360
[LightGBM] [Info] Start training from score 8.277824


100%|██████████| 40/40 [3:19:43<00:00, 299.58s/it]


In [6]:
models.to_csv( 'raw_data_LazyPredict.csv' )
models

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
ExtraTreesRegressor,0.69,0.69,1.67,783.85
RandomForestRegressor,0.67,0.67,1.71,2108.89
BaggingRegressor,0.63,0.63,1.81,219.22
XGBRegressor,0.62,0.62,1.85,4.39
LGBMRegressor,0.59,0.60,1.90,7.24
HistGradientBoostingRegressor,0.59,0.59,1.91,9.69
GradientBoostingRegressor,0.55,0.55,2.01,385.48
LassoLarsCV,0.51,0.51,2.09,8.18
LassoCV,0.51,0.51,2.09,19.54


### Approach 2: Imputing the Proccessed data ( long format ) + LazyPRedict

In [7]:
# Read the processed data
train2 = pd.read_csv( '../data/interim/all_train.csv' )

#Separate columns of interest
coi = ['bg','insulin','hr','steps','cals','bg+1:00']
data = train2[ coi ]

# Because of the way the long format data is generated, some NAs are introduced in thsi columns
# For this approach, we will remove those
data = data.dropna( subset = 'bg+1:00' )
data

,bg,insulin,hr,steps,cals,bg+1:00
71,15.10,0.04,NaN,NaN,NaN,13.40
74,14.40,0.04,NaN,NaN,NaN,12.80
77,13.90,0.04,NaN,NaN,NaN,15.50
80,13.80,0.04,NaN,NaN,NaN,14.80
83,13.40,0.04,NaN,NaN,NaN,12.70
...,...,...,...,...,...,...
235122,9.80,0.28,NaN,NaN,6.75,11.10
235123,10.30,1.06,NaN,NaN,6.75,10.90
235124,10.70,0.56,NaN,NaN,6.75,10.70
235125,11.00,0.05,NaN,NaN,6.75,10.50


In [8]:
# Do a train/Test split and impute columns with the medians of the Train set
X = data.drop( 'bg+1:00' , axis = 1 )
y = data['bg+1:00']
X_train , X_test , y_train , y_test = train_test_split( X , y , test_size = .3 , random_state = 42 )

medians = X_train.median( axis = 0 )
X_train = X_train.fillna( medians )
X_test = X_test.fillna( medians )

In [9]:
# Run a LazyPRedict Regressor
from lazypredict.Supervised import LazyRegressor
reg = LazyRegressor( verbose = 0 , ignore_warnings = False , custom_metric = None )
models , predictions = reg.fit( X_train , X_test , y_train , y_test )

 95%|█████████▌| 38/40 [1:21:54<04:25, 132.97s/it]  

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000410 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1230
[LightGBM] [Info] Number of data points in the train set: 123916, number of used features: 5
[LightGBM] [Info] Start training from score 8.277824


100%|██████████| 40/40 [1:21:55<00:00, 122.88s/it]


In [10]:
models.to_csv( 'LongFormat_data_MedianImp_LazyPredict_res.csv' )
models

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
XGBRegressor,0.53,0.53,2.05,0.42
LGBMRegressor,0.53,0.53,2.06,0.40
HistGradientBoostingRegressor,0.53,0.53,2.06,0.83
GradientBoostingRegressor,0.51,0.51,2.09,6.41
RandomForestRegressor,0.50,0.50,2.11,25.28
MLPRegressor,0.50,0.50,2.11,45.31
NuSVR,0.50,0.50,2.12,1370.76
SVR,0.49,0.49,2.13,420.90
RidgeCV,0.48,0.48,2.16,0.03


### Approach 3: Imputing the Proccessed data ( long format ) by time series interpolation + LazyPRedict

In [11]:
# Read the processed data
train2 = pd.read_csv( '../data/interim/all_train.csv' )

#Separate columns of interest
coi = ['bg','insulin','hr','steps','cals','bg+1:00']
data = train2[ coi ]

# Because of the way the long format data is generated, some NAs are introduced in thsi columns
# For this approach, we will remove those
data = data.dropna( subset = 'bg+1:00' )

# Use linear interpolation to impute NAs.
# USe backward- and forward- fill to account for boundaries, where no  imputation is possible
for col in ['bg','insulin','hr','steps','cals']:
    data[ col ] = data[ col ].interpolate( method = 'linear' ).bfill().ffill()

data

,bg,insulin,hr,steps,cals,bg+1:00
71,15.10,0.04,99.60,0.00,4.80,13.40
74,14.40,0.04,99.60,0.00,4.80,12.80
77,13.90,0.04,99.60,0.00,4.80,15.50
80,13.80,0.04,99.60,0.00,4.80,14.80
83,13.40,0.04,99.60,0.00,4.80,12.70
...,...,...,...,...,...,...
235122,9.80,0.28,104.40,8.00,6.75,11.10
235123,10.30,1.06,104.40,8.00,6.75,10.90
235124,10.70,0.56,104.40,8.00,6.75,10.70
235125,11.00,0.05,104.40,8.00,6.75,10.50


In [12]:
# Do a train/Test split a
X = data.drop( 'bg+1:00' , axis = 1 )
y = data['bg+1:00']
X_train , X_test , y_train , y_test = train_test_split( X , y , test_size = .3 , random_state = 42 )

In [13]:
# Run a LazyPredict Regressor
from lazypredict.Supervised import LazyRegressor
reg = LazyRegressor( verbose = 0 , ignore_warnings = False , custom_metric = None )
models , predictions = reg.fit( X_train , X_test , y_train , y_test )

 95%|█████████▌| 38/40 [19:36<02:13, 66.67s/it] 

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000368 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 123916, number of used features: 5
[LightGBM] [Info] Start training from score 8.277824


100%|██████████| 40/40 [19:37<00:00, 29.44s/it]


In [14]:
models.to_csv( 'LongFormat_data_InterpolationImp_LazyPredict_res.csv' )
models

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
RandomForestRegressor,0.58,0.58,1.93,36.08
XGBRegressor,0.56,0.56,1.98,0.42
LGBMRegressor,0.55,0.55,2.02,0.52
BaggingRegressor,0.54,0.54,2.02,3.27
HistGradientBoostingRegressor,0.54,0.54,2.02,1.04
GradientBoostingRegressor,0.52,0.52,2.07,8.62
ExtraTreesRegressor,0.52,0.52,2.07,12.26
MLPRegressor,0.52,0.52,2.08,48.93
NuSVR,0.51,0.51,2.09,401.19


### Summary
* Running a LazyRegressor task on the raw data ( median imputed ) has shown that tree based methods, like RandomForestRegressor , BaggingRegressor , XGBRegressor  and LGBMRegressor are the best suited for predicting **bg+1:00** 
* For a LazyRegressor task, better results were achieved using the raw data ( median imputed ), which contains lag-features, than using the entire series  iin a long format